Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Automated Machine Learning
_**Text Classification Using Deep Learning**_

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Evaluate](#Evaluate)

## Introduction
This notebook demonstrates classification with text data using deep learning in AutoML.

AutoML highlights here include using deep neural networks (DNNs) to create embedded features from text data. Depending on the compute cluster the user provides, AutoML tried out Bidirectional Encoder Representations from Transformers (BERT) when a GPU compute is used, and Bidirectional Long-Short Term neural network (BiLSTM) when a CPU compute is used, thereby optimizing the choice of DNN for the uesr's setup.

Make sure you have executed the [configuration](../../../configuration.ipynb) before running this notebook.

An Enterprise workspace is required for this notebook. To learn more about creating an Enterprise workspace or upgrading to an Enterprise workspace from the Azure portal, please visit our [Workspace page](https://docs.microsoft.com/azure/machine-learning/service/concept-workspace#upgrade).

Notebook synopsis:
1. Creating an Experiment in an existing Workspace
2. Configuration and remote run of AutoML for a text dataset (20 Newsgroups dataset from scikit-learn) for classification
3. Registering the best model for future use
4. Evaluating the final model on a test set

## Setup

In [ ]:
import logging
import os
import shutil

import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.run import Run
from azureml.widgets import RunDetails
from azureml.core.model import Model 
from helper import run_inference, get_result_df
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.featurization import FeaturizationConfig
from sklearn.model_selection import train_test_split

This sample notebook may use features that are not available in previous versions of the Azure ML SDK.

In [ ]:
print("This notebook was created using version 1.34.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

As part of the setup you have already created a <b>Workspace</b>. To run AutoML, you also need to create an <b>Experiment</b>. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

In [ ]:
subscription_id = '<your subscription_id>'
resource_group = '<your resource_group>'
workspace_name = '<your workspace_name>'

ws = Workspace(subscription_id, resource_group, workspace_name)

experiment_name = 'livedoor-news-classification'
experiment = Experiment(ws, experiment_name)

## Set up a compute cluster
This section uses a user-provided compute cluster (named "dnntext-cluster" in this example). If a cluster with this name does not exist in the user's workspace, the below code will create a new cluster. You can choose the parameters of the cluster as mentioned in the comments.

Whether you provide/select a CPU or GPU cluster, AutoML will choose the appropriate DNN for that setup - BiLSTM or BERT text featurizer will be included in the candidate featurizers on CPU and GPU respectively.  If your goal is to obtain the most accurate model, we recommend you use GPU clusters since BERT featurizers usually outperform BiLSTM featurizers.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster.
amlcompute_cluster_name = "gpucluster24"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC24rs_V3", 
                                                               # CPU for BiLSTM, such as "STANDARD_D2_V2" 
                                                           # To use BERT (this is recommended for best performance), select a GPU such as "STANDARD_NC24rs_V3" 
                                                           # or similar GPU option available in your workspace
                                                           min_nodes = 0,
                                                           max_nodes = 10,
                                                           vm_priority='lowpriority') ## vm_priority='lowpriority' | 'dedicated'
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

### Get data
For this notebook we will use 20 Newsgroups data from scikit-learn. We filter the data to contain four classes and take a sample as training data. Please note that for accuracy improvement, more data is needed. For this notebook we provide a small-data example so that you can use this template to use with your larger sized data.

In [ ]:
data_dir = "data" # Local directory to store data
blobstore_datadir = data_dir # Blob store directory to store data in

target_column_name = 'label'
df = Dataset.get_by_name(ws, name='livedoor-news')
df = df.to_pandas_dataframe()
df.drop(labels=['url', 'date'], axis=1, inplace=True)

data_train, data_test = train_test_split(df, test_size=0.25, random_state=0)

# Only checking the loaded dataset
print ('** train - columns:')
print (data_train.columns)
'''
print ('** train - dataset:')
print (data_train)
print ('** test - dataset:')
print (data_test)
'''

#### Fetch data and upload to datastore for use in training for Remote Compute (AmlCompute)

In [ ]:
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)
    
train_data_fname = data_dir + '/train_data.csv'
test_data_fname = data_dir + '/test_data.csv'

data_train.to_csv(train_data_fname, index=False)
data_test.to_csv(test_data_fname, index=False)

datastore = ws.get_default_datastore()
datastore.upload(src_dir=data_dir, target_path=blobstore_datadir,
                    overwrite=True)

In [ ]:
train_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, blobstore_datadir + '/train_data.csv')])

### Prepare AutoML run

This step requires an Enterprise workspace to gain access to this feature. To learn more about creating an Enterprise workspace or upgrading to an Enterprise workspace from the Azure portal, please visit our [Workspace page](https://docs.microsoft.com/azure/machine-learning/service/concept-workspace#upgrade).

Reference this page [実験の設定を構成する](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings) for Configration detail. 

blocking_model name: [azureml.train.automl.constants.SupportedModels.Classification](https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.constants.supportedmodels.classification?view=azure-ml-py)

SupportedTranformers name is here: [SupportedTransformers](https://docs.microsoft.com/en-us/python/api/azureml-automl-core/azureml.automl.core.constants.supportedtransformers?view=azure-ml-py)

In [ ]:
featurization_config = FeaturizationConfig(dataset_language='jpn')
#featurization_config.blocked_transformers = ['TfIdf','CountVectorizer']

automl_settings = {
    'experiment_timeout_minutes': 60,
    'primary_metric' : 'AUC_weighted', #'AUC_weighted', 'accuracy'
    'experiment_exit_score': 0.9,
    'max_concurrent_iterations': 4,
    'max_cores_per_iteration': -1,
    'enable_dnn': True,
    'enable_early_stopping': True,
    'force_text_dnn': True, # enable BERT featurization
    'validation_size': 0.15,
    'verbosity': logging.INFO,
    'featurization': featurization_config,
    'enable_voting_ensemble': False, # this cut the final ensumble job
    'enable_stack_ensemble': False   # this cut the final ensumble job
#    'iterations': 2 ## mainly for DEBUG: Test purpose to stop job earlier to undestand AutoMLConfig parameters behavior or so.
}

automl_config = AutoMLConfig(task              = 'classification',
                             debug_log         = 'automl_debug.log',
                             compute_target    = compute_target,
                             training_data     = train_dataset,
                             label_column_name = target_column_name,
                             **automl_settings
                            )

#### Submit AutoML Run

If you want to see running state more graphical output of AutoML job.
1) please disble show_output like this. 
```
automl_run = experiment.submit(automl_config)
```
2) run the following code after experiment.submit(automl_config)
```
RunDetails(automl_run).show()
```

In [ ]:
#automl_run = experiment.submit(automl_config)
automl_run = experiment.submit(automl_config, show_output=True)

In [ ]:
#RunDetails(automl_run).show() --- it shows more graphical training status.

In [ ]:
#automl_run.wait_for_completion()

Displaying the run objects gives you links to the visual tools in the Azure Portal. Go try them!

### Retrieve the Best Model
Below we select the best model pipeline from our iterations, use it to test on test data on the same compute cluster.

You can test the model locally to get a feel of the input/output. When the model contains BERT, this step will require pytorch and pytorch-transformers installed in your local environment. The exact versions of these packages can be found in the **automl_env.yml** file located in the local copy of your MachineLearningNotebooks folder here:
MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/automl_env.yml

In [ ]:
best_run, fitted_model = automl_run.get_output()

You can now see what text transformations are used to convert text data to features for this dataset, including deep learning transformations based on BiLSTM or Transformer (BERT is one implementation of a Transformer) models.

In [ ]:
text_transformations_used = []
for column_group in fitted_model.named_steps['datatransformer'].get_featurization_summary():
    text_transformations_used.extend(column_group['Transformations'])
text_transformations_used

### Registering the best model
We now register the best fitted model from the AutoML Run for use in future deployments.  

Get results stats, extract the best model from AutoML run, download and register the resultant best model

In [ ]:
summary_df = get_result_df(automl_run)
best_dnn_run_id = summary_df['run_id'].iloc[0]
best_dnn_run = Run(experiment, best_dnn_run_id)

In [ ]:
model_dir = 'Model' # Local folder where the model will be stored temporarily
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)
    
best_dnn_run.download_file('outputs/model.pkl', model_dir + '/model.pkl')

Register the model in your Azure Machine Learning Workspace. If you previously registered a model, please make sure to delete it so as to replace it with this new model.

In [ ]:
# Register the model
model_name = 'textDNN-article'
model = Model.register(model_path = model_dir + '/model.pkl',
                       model_name = model_name,
                       tags=None,
                       workspace=ws)

## Evaluate on Test Data

We now use the best fitted model from the AutoML Run to make predictions on the test set.  

Test set schema should match that of the training set.

In [ ]:
test_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, blobstore_datadir + '/test_data.csv')])

# preview the first 3 rows of the dataset
test_dataset.take(3).to_pandas_dataframe()

In [ ]:
test_experiment = Experiment(ws, experiment_name + "_test")

In [ ]:
script_folder = os.path.join(os.getcwd(), 'inference')
os.makedirs(script_folder, exist_ok=True)
shutil.copy('infer.py', script_folder)

In [ ]:
test_run = run_inference(test_experiment, compute_target, script_folder, best_dnn_run,
                         train_dataset, test_dataset, target_column_name, model_name)

Display computed metrics

In [ ]:
test_run

In [ ]:
RunDetails(test_run).show()

In [ ]:
test_run.wait_for_completion()

In [ ]:
pd.Series(test_run.get_metrics())